In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd

## Merge NBER crosswalk and Alex's population table

In [2]:
crosswalk = pd.read_csv("./cbsa_county_crosswalk.csv")

In [3]:
def wrangle_crosswalk(df):
    df = df.assign(
        fipsstatecode = df.fipsstatecode.astype(str).str.pad(width=2, side="left", fillchar="0"),
        fipscountycode = df.fipscountycode.astype(str).str.pad(width=3, side="left", fillchar="0"),
        state = df.statename,
        county = df.countycountyequivalent,
        metro_micro = df.metropolitanmicropolitanstatis,
    )
    
    df["county_fips"] = df.fipsstatecode.map(str) + df.fipscountycode.map(str)
    
    keep_col = [
        "cbsacode",
        "cbsatitle",
        "metro_micro",
        "county",
        "state",
        "county_fips",
        "fipsstatecode",
        "fipscountycode"
    ]
    
    df = df[keep_col]
    
    return df

crosswalk = wrangle_crosswalk(crosswalk)

In [4]:
# Merge in population
pop = pd.read_csv("./population.csv")

pop = pop.assign(
    county_fips = pop.FIPS.astype(str).str.pad(width=5, side="left", fillchar="0"),
    population = pop.Metro_Area_Population
)

pop = pop[['county_fips', 'population']]

In [5]:
df = pd.merge(crosswalk, pop, on = 'county_fips', how = 'left', validate = '1:1')

df.to_csv('s3://public-health-dashboard/jhu_covid19/msa_county_pop_crosswalk.csv', index=False)

## Bring in JHU data

In [6]:
# Switch pop crosswalk with GitHub URL, once we upload it there
pop = pd.read_csv('s3://public-health-dashboard/jhu_covid19/msa_county_pop_crosswalk.csv',
                 dtype={"county_fips":"str", 
                        "cbsacode":"str"})

pop = pop[["cbsacode", "cbsatitle", "population", "county_fips"]]

In [7]:
# Substitute for feature layer
url = "https://services5.arcgis.com/7nsPwEMP38bSkCjy/arcgis/rest/services/county_time_series_331/FeatureServer/0/query?where=1%3D1&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&returnGeodetic=false&outFields=county%2C+state%2C+fips%2C+date%2C+Lat%2C+Lon%2C+cases%2C+deaths%2C+incident_rate%2C+people_tested%2C+state_cases%2C+state_deaths%2C+new_cases%2C+new_deaths%2C+new_state_cases%2C+new_state_deaths&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=pgeojson&token="
gdf = gpd.read_file(url)

In [8]:
def subset_msa(df):
    # 5 MSAs to plot: NYC, SF_SJ, SEA, DET, LA
    df = df[
        df.cbsatitle.str.contains("Los Angeles") |
        df.cbsatitle.str.contains("New York") |
        df.cbsatitle.str.contains("San Francisco") |
        df.cbsatitle.str.contains("San Jose") |
        df.cbsatitle.str.contains("Seattle") |
        df.cbsatitle.str.contains("Detroit")
    ]
    
    def new_categories(row):
        if ("San Francisco" in row.cbsatitle) or ("San Jose" in row.cbsatitle):
            return "SF/SJ"
        elif "Los Angeles" in row.cbsatitle:
            return "LA/OC"
        elif "New York City" in row.cbsatitle:
            return "NYC"
        elif "Seattle" in row.cbsatitle:
            return "SEA"
        elif "Detroit" in row.cbsatitle:
            return "DET"
        
    df = df.assign(
        msa = df.apply(new_categories, axis=1)
    )
    
    return df

pop = subset_msa(pop)

In [9]:
def merge_jhu_with_msa_crosswalk(gdf):
    # Merge in population
    gdf = pd.merge(gdf, pop, left_on="fips", right_on="county_fips", how="inner", validate="m:1")
    
    # Aggregate by MSA
    group_cols = ["cbsacode", "msa", "population", "date"]
    msa = gdf.groupby(group_cols).agg({
        "cases":"sum",
        "deaths":"sum"
    }).reset_index()
    
    # Calculate rate per 1M
    rate = 1000000
    msa = msa.assign(
        cases_per_1M = msa.cases / msa.population * rate,
        deaths_per_1M = msa.deaths / msa.population * rate,
    )
    
    return msa

In [11]:
msa = merge_jhu_with_msa_crosswalk(gdf)
msa.head()

,cbsacode,msa,population,date,cases,deaths,cases_per_1M,deaths_per_1M
0,19820,DET,4319629.0,1583823600000,2.0,0.0,0.463003,0.0
1,19820,DET,4319629.0,1583910000000,2.0,0.0,0.463003,0.0
2,19820,DET,4319629.0,1583996400000,5.0,0.0,1.157507,0.0
3,19820,DET,4319629.0,1584082800000,14.0,0.0,3.241019,0.0
4,19820,DET,4319629.0,1584169200000,20.0,0.0,4.630027,0.0
